# 1.3 Data Preparation
After obtaining a dataset, we'll often need to prepare it for analysis. We'll go over a few key data preparation tasks.

## Counting and Sorting
Counting and sorting are a great first step to analyzing data. We can verify that the data is complete, and ensure there's no missing values. This also allows us to see the range of values for all of our variables.

In [27]:
""" EXAMPLE 1.3

We'll analyze the information of BalanceGig, a company that matches independent workers
for short-term engagements with businesses. The data are:

-> Wage: the hourly wage
-> Industry: the client's industry
-> Job: the employee's job classification 
"""
import pandas as pd

# Read the data.
data = pd.read_csv('Gig.csv', index_col=False)
data.head(5)

,EmployeeID,Wage,Industry,Job
0,1,32.81,Construction,Analyst
1,2,46.00,Automotive,Engineer
2,3,43.13,Construction,Sales Rep
3,4,48.09,Automotive,Other
4,5,43.62,Automotive,Accountant


The manager suspects that this data is incomplete. We want to find the number of missing observations for each variable. We'd also like information on the number of employees who:
1) Worked in the automotive industry
2) Earned more than $\$30$ per hour
3) Worked in the automotive industry and earned more than $\$30$ per hour

And finally, the manager wants to know the hourly wage of lowest- and the highest-paid employees at the company as a whole and the hourly wage of the lowest- and the highest-paid accountants who worked in the automotive and the tech industries.

In [28]:
# Let's see how many missing variables there are for each variable.
data.isnull().sum()

EmployeeID     0
Wage           0
Industry      10
Job           16
dtype: int64

We have $10$ missing values for Industry, and $16$ missing values for Job.

In [29]:
# How many gig workers were in the automotive industry?
auto_mask = data['Industry'] == 'Automotive'
count_auto = sum(auto_mask)

# How many gig workers earned more than $30 per hour?
wage_mask = data['Wage'] > 30
count_30 = sum(wage_mask)

# How many worked in the automotive industry and earned more than $30 per hour?
auto_wage_mask = auto_mask & wage_mask
count_both = sum(auto_wage_mask)

print('Workers in automotive:', count_auto)
print('Workers earned more than $30:', count_30)
print('Worked in auto and earned more than $30:', count_both)


Workers in automotive: 190
Workers earned more than $30: 536
Worked in auto and earned more than $30: 181


In [30]:
# Find hourly wage of the lowest and highest paid employees.
data['Wage'].min(), data['Wage'].max()

(24.28, 51.0)

In [31]:
# Find hourly wage of the lowest and highest paid accountants in automotive and tech.
ind_mask_auto = (data['Industry'] == 'Automotive') & (data['Job'] == 'Accountant')
ind_mask_tech = (data['Industry'] == 'Tech') & (data['Job'] == 'Accountant')


auto_wage_range = data[ind_mask_auto]['Wage'].min(), data[ind_mask_auto]['Wage'].max()
tech_wage_range = data[ind_mask_tech]['Wage'].min(), data[ind_mask_tech]['Wage'].max()

print('Wage range for auto:', auto_wage_range)
print('Wage range for tech:', tech_wage_range)

Wage range for auto: (28.74, 49.32)
Wage range for tech: (36.13, 49.49)


### A Note on Handling Missing Values
There are two common strategies for missing values. The first is the **omission** strategy, where we exclude these observations from the analysis. The second is the **imputation** strategy, where we replace the missing valeus with some reasonable imputation. 

For example, for numerical variables we may impute missing information with the average value in the dataset. Or for a categorical variable, we may impute the most dominant category.

## Subsetting
Extracting portions of a dataset that are relevant is called **subsetting**. For example, a multinational corporation may want to analyze data on its sales in Germany. Thus, we'd create a subset of the data where the sales were conducted in Germany.

In [32]:
""" EXAMPLE 1.4

We want a better understanding of Organic Food Superstore's customers who are
college-educated millennials, born between 1982 and 2000. We believe sex, household
size, annual income, total spending, total number of orders, and channel through
which the customer was acquired are useful features.
"""
# Read the data.
customers = pd.read_csv('Customers.csv', index_col=False)
customers.head(5)

,CustID,Sex,Race,BirthDate,College,HHSize,Income,Spending,Orders,Channel
0,1530016,Female,Black,12/16/1986,Yes,5,53000,241,3,SM
1,1531136,Male,White,5/9/1993,Yes,5,94000,843,12,TV
2,1532160,Male,Black,5/22/1966,Yes,2,64000,719,9,TV
3,1532307,Male,White,9/16/1964,Yes,4,60000,582,13,SM
4,1532356,Female,Hispanic,7/15/1964,No,5,47000,845,7,Web


In [33]:
# Identity college-educated customers with mask.
college_mask = customers['College'] == 'Yes'

# Get Birth year for comparison, make mask
customers['BirthYear'] = customers['BirthDate'].str.strip().str[-4:].astype(int)
birthyr_mask = (1999 >= customers.BirthYear)&(customers.BirthYear >= 1982)

# Select college educ. millennials.
collmil_customers = customers[college_mask & birthyr_mask].reset_index(drop=True)

# Drop irrelevant cols (CustID, Race, BirthDate, College)
collmil_customers.drop(['CustID', 'Race', 'BirthDate', 'College', 'BirthYear'], axis=1, inplace=True)

collmil_customers.head(5)

,Sex,HHSize,Income,Spending,Orders,Channel
0,Female,5,53000,241,3,SM
1,Male,5,94000,843,12,TV
2,Female,3,84000,153,2,Web
3,Male,1,97000,1028,17,Web
4,Female,3,64000,915,15,Referral


In [34]:
# Female college-educated millennials.
fem_mask = collmil_customers['Sex'] == 'Female'
fem_data = collmil_customers[fem_mask].reset_index(drop=True)
fem_data.head(5)

,Sex,HHSize,Income,Spending,Orders,Channel
0,Female,5,53000,241,3,SM
1,Female,3,84000,153,2,Web
2,Female,3,64000,915,15,Referral
3,Female,3,42000,313,4,TV
4,Female,5,97000,911,16,Web


In [35]:
# Male college-educated millennials.
male_mask = collmil_customers['Sex'] == 'Male'
male_data = collmil_customers[male_mask].reset_index(drop=True)
male_data.head(5)

,Sex,HHSize,Income,Spending,Orders,Channel
0,Male,5,94000,843,12,TV
1,Male,1,97000,1028,17,Web
2,Male,2,114000,665,7,TV
3,Male,2,94000,524,7,Referral
4,Male,1,91000,800,10,Web
